## Data Sources
*   Downloaded Dataset Source: https://catalog.data.gov/dataset/heart-disease-mortality-data-among-us-adults-35-by-state-territory-and-county
*   Web Collection #1 Source: https://en.wikipedia.org/wiki/Obesity_in_the_United_States
*   Web Collection #2 Source: https://www.healthit.gov/data/open-api?source=Surescripts_04-2014_State.csv



In [112]:
import numpy as np
import random as random
import pandas as pd
import json, csv, requests, re
from bs4 import BeautifulSoup

## Downloaded Dataset


In [119]:
def data_parser():
    #import pandas as pd  #already imported delete later
    df = pd.read_csv('Heart_Disease_Mortality.csv')

    #removes the columns
    df.drop('Year', inplace=True, axis=1)
    df.drop('GeographicLevel', inplace=True, axis=1)
    df.drop('DataSource', inplace=True, axis=1)
    df.drop('Class', inplace=True, axis=1)
    df.drop('Topic', inplace=True, axis=1)
    df.drop('Data_Value_Unit', inplace=True, axis=1)
    df.drop('Data_Value_Type', inplace=True, axis=1)
    df.drop('Data_Value_Footnote_Symbol', inplace=True, axis=1)
    df.drop('Data_Value_Footnote', inplace=True, axis=1)
    df.drop('StratificationCategory1', inplace=True, axis=1)
    df.drop('Stratification1', inplace=True, axis=1)
    df.drop('StratificationCategory2', inplace=True, axis=1)
    df.drop('Stratification2', inplace=True, axis=1)
    df.drop('TopicID', inplace=True, axis=1)
    df.drop('LocationID', inplace=True, axis=1)
    df.drop('Location 1', inplace=True, axis=1)
    df.drop('LocationDesc', inplace=True, axis=1)

    #getting rid of the columns with np.nan in the 'Data_Value' column
    df = df[df['Data_Value'].notna()]

    #sorting the datavalues by 'LocationAbbr'
    df.sort_values(by=['LocationAbbr'], ascending=True, inplace = True)

    #resets the index of each row to increase from 0 to the number of rows
    df.reset_index(inplace = True, drop = True)

    #Finding the average number of deaths due to Heart Disease per state/territory per 100,000 people
    df = df.groupby(['LocationAbbr']).agg({'Data_Value': 'mean'})

    #Rename the Data_Value column to Avg_Num_Deaths
    df.rename(columns = {'Data_Value':'Avg_Num_Deaths'}, inplace = True)

    #Finding the total number of deaths within in the US states and territories, this can be used to find the percentage that each state makes up
    total = 0
    for row in df['Avg_Num_Deaths']:
        total += row

    #Adding a new column to df that represents the percentage of heart disease deaths in the country that fall within the state
    percentage  = []
    for row in df['Avg_Num_Deaths']:
        percentage.append(row/total*100)
    df['Percent Total %'] = percentage

    #Writing the dataframe to an output CSV file
    df.to_csv('heart_disease_output.csv', index=True)

    print(df)

############ Function Call ############
data_parser()

              Avg_Num_Deaths  Percent Total %
LocationAbbr                                 
AK                318.396517         1.698614
AL                452.940996         2.416396
AR                447.272996         2.386157
AS                343.533333         1.832716
AZ                272.962357         1.456227
CA                298.068489         1.590166
CO                247.797399         1.321975
CT                255.655882         1.363899
DC                306.178571         1.633432
DE                270.705172         1.444185
FL                286.450326         1.528184
GA                380.160799         2.028120
GU                529.800000         2.826431
HI                300.981707         1.605708
IA                329.146016         1.755962
ID                278.923204         1.488028
IL                343.175380         1.830807
IN                354.644851         1.891995
KS                339.465344         1.811014
KY                431.482062      

## Web Collection \#1


In [117]:
def web_parser1():
    #Url of the website with the table
    r = requests.get("https://en.wikipedia.org/wiki/Obesity_in_the_United_States")

    #Create a BeautifulSoup object
    soup = BeautifulSoup(r.text,"html.parser")

    #Find the table tag
    table = soup.find('table',{'class':'wikitable sortable'})

    #This is the list where the final data will be appended
    output_list = []

    #Find the header row and getting the column headers (tr), appending it to the final list
    header_row = table.find('tr')
    headers = header_row.find_all('th')
    header_info = []
    for header in headers:
        info = (header.get_text().strip())
        if "[" in info:
            info = info[:-4]
        if "[" in info:
            info = info[:-4]
        header_info.append(info)
    output_list.append(header_info)

    #Find all row tags (tr) in the table
    rows = table.find_all('tr')

    #This is the regex match text
    regex_replace = re.compile(r'\[.*?\]')

    #Iterate over each row and getting the data from each cell (td)
    for row in rows[1:]: #skipping the header row which would otherwise be blank
        info = row.find_all('td')
        row_data = []
        for item in info:
            cleaned_data = (item.get_text().strip())
            if "[" in cleaned_data:
                cleaned_data = cleaned_data[:-4]
            if "[" in cleaned_data:
                cleaned_data = cleaned_data[:-4]
            row_data.append(cleaned_data)
        output_list.append(row_data)

    #Replace the '-' with n/a to better represent the meaning of '-'
    for lists in output_list:
        for i in range(len(lists)):
            if lists[i] == '—':
                lists[i] = "n/a"
    
    #Writing the output_list to an output_file using csv
    #opening an output file for writing named US_Obesity
    with open('US_Obesity.csv', 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerows(output_list)
    
    print(output_list)

############ Function Call ############
web_parser1()

[['States, district, & territories', 'Obesity rank', 'Obese adults (mid-2000s)', 'Obese adults (2020)', 'Overweight (incl. obese) adults(mid-2000s)', 'Obese children and adolescents(mid-2000s)'], ['Alabama', '5', '30.1%', '36.3%', '65.4%', '16.7%'], ['Alaska', '9', '27.3%', '34.2%', '64.5%', '11.1%'], ['American Samoa', 'n/a', 'n/a', '75%', '95%', '35%'], ['Arizona', '30', '23.3%', '29.5%', '59.5%', '12.2%'], ['Arkansas', '7', '28.1%', '35.0%', '64.7%', '16.4%'], ['California', '48', '23.1%', '25.1%', '59.4%', '13.2%'], ['Colorado', '51', '21.0%', '22.6%', '55.0%', '9.9%'], ['Connecticut', '42', '20.8%', '26.9%', '58.7%', '12.3%'], ['Delaware', '23', '25.9%', '31.8%', '63.9%', '22.8%'], ['District of Columbia', '50', '22.1%', '23.0%', '55.0%', '14.8%'], ['Florida', '35', '23.3%', '28.4%', '60.8%', '14.4%'], ['Georgia', '24', '27.5%', '31.6%', '63.3%', '16.4%'], ['Guam', 'n/a', 'n/a', '28.3%', 'n/a', '22%'], ['Hawaii', '49', '20.7%', '23.8%', '55.3%', '13.3%'], ['Idaho', '32', '24.6%', 

## Web Collection \#2

In [116]:
def web_parser2():
    #making the initial API request
    url = "https://www.healthit.gov/data/open-api?source=Surescripts_04-2014_State.csv"
    r = requests.get(url).json()
    
    #removing the sub-dictionaries with the value "National" or null values for the key "region"
    #iterating backwards because doing it forwards means that removing a sub-dictionary shifts the indices of the subsequent sub-dictionaries by 1, meaning some dictionaries that should be deleted won't be deleted
    for num in range(len(r)-1, -1, -1):
        if r[num]["region"] == "National" or r[num]["region"] == "":
            r.remove(r[num])
    
    #removing the sub-dictionaries with only the year and no month for the "period" key
    #iterating backwards because doing it forwards means that removing a sub-dictionary shifts the indices of the subsequent sub-dictionaries by 1, meaning some dictionaries that should be deleted won't be deleted
    for num in range(len(r)-1, -1, -1):
        match_object = re.match("[0-9]{4}-[0-9]{2}", r[num]["period"]) #"period" key is in a "YYYY-MM" format for periods including month and year and "YYYY" for periods including only year
        if match_object == None:
            r.remove(r[num])
    
    #replacing the remaining dates with their years only (deleting their attached months)
    #this will make it so we can group by year when the data is converted into a pandas dataframe
    for dictionary in r:
        dictionary["period"] = re.sub("[0-9]{4}-[0-9]{2}", dictionary["period"][0:4], dictionary["period"])
    
    #removing the "region" key from the sub-dictionaries
    for dictionary in r:
        dictionary.pop("region")
        
    #converting the json response object into a pandas dataframe
    df_v1 = pd.DataFrame(r)

    #dropping the unnecessary columns that won't be used in data analysis
    df_v1.drop(["tot_e_Rx_thru_ehr", "tot_e_Rx_thru_stand_alone",  "tot_primary_care_e_Rx", "tot_non_primary_care_e_Rx", "tot_nurse_prac_e_Rx", "tot_phys_asst_e_Rx", "pct_new_renewal_e_Rx", "tot_med_hist_reqs",  "tot_med_hist_req_resps"], axis = 1, inplace = True)
    
    #removing the sub-dictionaries with empty strings for "pct_pharm_enabled" and "pct_pharm_e_Rx" columns
    df_v2 = df_v1.replace("", np.nan)
    df_v2.dropna(axis = 0, inplace = True)
    
    #converting the types of necessary attributes from str to int/float
    df_v2["tot_e_Rx"] = df_v2["tot_e_Rx"].astype("int")
    df_v2["pct_pharm_enabled"] = df_v2["pct_pharm_enabled"].astype("float")
    df_v2["pct_pharm_e_Rx"] = df_v2["pct_pharm_e_Rx"].astype("float")
    
    #grouping the dataframe by "region_code" and then by "period", and then finding the means of the remaining columns
    df_v3 = df_v2.groupby(["region_code", "period"]).agg({"tot_e_Rx": "mean", "pct_pharm_enabled": "mean", "pct_pharm_e_Rx": "mean"})
    
    #writing the resulting dataframe to a csv file
    df_v3.to_csv("electronic-prescribing-adoption-use.csv", index = True)
    
    print(df_v3)
    
############ Function Call ############
web_parser2()

                       tot_e_Rx  pct_pharm_enabled  pct_pharm_e_Rx
region_code period                                                
AK          2008     113.000000           0.780000        0.710000
            2009     280.000000           0.890000        0.830000
            2010     386.500000           0.890000        0.885000
            2011     551.583333           0.925000        0.923333
            2012     765.416667           0.940000        0.939167
...                         ...                ...             ...
WY          2010     259.000000           0.855000        0.830000
            2011     423.500000           0.913333        0.905833
            2012     662.250000           0.953333        0.950833
            2013     900.333333           0.972500        0.972500
            2014    1005.500000           0.975000        0.970000

[367 rows x 3 columns]


# Inconsistencies

1. CSV np.nan inconsistency: In the CSV file, after turning the file into a pandas dataframe, there were quite a few data values that registered as np.nan. With np.nan values, we would not be able to find the average values of deaths per state. To deal with this, we removed all rows that contained np.nan in the Data_Values column using masking and isna(). This left us with a dataframe with only rows that have values in the Data_Values column so we were able to actually find the averages number of deaths related to heart disease per state.

2. BS4 footnote inconsistency: In the Wikipedia table, in both the header and some of the data values, there were footnotes written as "\[\_\_\]" where "\_\_" represents some number. This posed a lot of issues regarding using the actual data while creating visualizations. We dealt with this by using string methods to remove the “\[\_\_\]” for each item using a for loop to shorten the text while the “\[“ was still present. This way, for phase 3 it will be easier to compare the data with the csv data and visualise it.

3. JSON/API empty string inconsistency: There were empty strings for several values in the API, meaning that their corresponding json dictionaries were rendered useless for data analysis purposes. Those json dictionaries with empty strings were removed by converting the original response object to a pandas dataframe, changing the empty strings to NaN values, and then using the dropna function to drop rows with NaN values.
